# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.48.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

I'm using the heart failure prediction dataset fromm kaggle (https://www.kaggle.com/datasets/andrewmvd/heart-failure-clinical-data?resource=download). This dataset contains 12 features such as age, diabetes, high blood pressure etc. It's a classification task to predict mortality bay heart failure based on these features. People with a high cardiovascular risk due to the presence of risk factors (detected by this ML model) can prevent a heart failure by changing some behavior.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'heart-failure-experiment'

experiment=Experiment(ws, experiment_name)

# creating compute cluster

amlcompute_cluster_name = "experiment-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)


InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


In [4]:
#get data: load from workspace
found = False
key = "kaggle-heart-failure-prediction"
description_text = "Heart Failure Prediction dataset from kaggle"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://mlstrg225794.blob.core.windows.net/azureml-blobstore-bfad7455-0114-4658-93ad-3eb28d482c62/UI/2023-02-13_103739_UTC/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

I chose 'accurancy' as primary metric because it is a good meassure of a classification task. In addition, it is easier to compare the both models later without having a look at all metrics (the model with the bist accuracy should be deployed). To ensure that the training of the model does not take too long (due to costs), I decided that the experiment should time out after 20 minutes and maximum 5 iterations to test in the training job. this number should be not greater than the number of nodes in the compute cluster.

Here, it is a classification task to predict the column 'DEATH_EVENT' in the dataset. The early stopping is enabled to save unnecessary running time and to ensure that the best accuracy is reached. The automatic featurization is enabled, so data guardrails and feauturization steps are done automatically in the preprocessing.

In [5]:
# TODO: Put your automl settings here
project_folder = './prediction-project'

automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [6]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-experiment,AutoML_83ce889d-2458-47f2-8f90-b90abe612f43,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [8]:
remote_run.wait_for_completion()

{'runId': 'AutoML_83ce889d-2458-47f2-8f90-b90abe612f43',
 'target': 'experiment-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-13T11:10:51.290178Z',
 'endTimeUtc': '2023-02-13T11:34:43.550145Z',
 'services': {},
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'experiment-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-experiment","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-225794","workspace_name":"quick-starts-ws-225794","region":"westeurope","compute_target":"experiment-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_compl

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run_automl, fitted_model_automl = remote_run.get_output()
print('Id of best run is: ' + str(best_run_automl.id))
RunDetails(best_run_automl).show()

Id of best run is: AutoML_83ce889d-2458-47f2-8f90-b90abe612f43_39


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [23]:
#TODO: Save the best model
import pickle

with open('./model.pkl', "rb" ) as f:
    best_model = pickle.load(f)
best_model

#model is already saved under ./model.pkl

EOFError: Ran out of input

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [21]:
#register model
best_run_automl.register_model(
    model_name='best-model-automl', 
    model_path='outputs/model.pkl', 
    description='the best model from auto ml run to predict heart failure')

best_run_automl.download_files(output_directory='./best_model-automl-output')

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [31]:
#Create inference config and deploy model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model

#load registered model
model = Model(ws, 'best-model-automl')

#load yaml file
env = Environment.from_conda_specification("best-model-env", "best_model-automl-output/outputs/conda_env_v_1_0_0.yml")

#create inference config
inference_config = InferenceConfig(environment=env, 
                                   source_directory='./best_model-automl-output/outputs', 
                                   entry_script='./scoring_file_v_2_0_0.py')

#create deployment config/ deployment as webservice with enabled insights
deployment_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    enable_app_insights=True,
    auth_enabled=True) 

#deploy best model 
endpoint = Model.deploy(ws, "best-model-automl-endpoint", [model], inference_config, deployment_config, overwrite=True)
endpoint.wait_for_deployment(show_output = True)

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-02-13 13:05:50+00:00 Creating Container Registry if not exists.

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


.
2023-02-13 13:15:51+00:00 Registering the environment.
2023-02-13 13:15:52+00:00 Building image.

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Usin

.
2023-02-13 13:36:17+00:00 Generating deployment configuration..
2023-02-13 13:36:18+00:00 Submitting deployment to compute..
2023-02-13 13:36:24+00:00 Checking the status of deployment best-model-automl-endpoint.

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads
INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


.
2023-02-13 13:38:39+00:00 Checking the status of inference endpoint best-model-automl-endpoint.

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads



Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, send a request to the web service you deployed to test it.

In [35]:
import json
import requests

#scoring uri and key
scoring_uri = "http://96ed3b1a-a970-4114-861d-caa73bfe885d.westeurope.azurecontainer.io/score"
key = "PyjT64SSfQhOWZCQlw9jPeM89LhmCWcd"
data = {
    "Inputs":{
        "data": [
            {
                "age": 65,
                "anaemia": 1,
                "creatinine_phosphokinase": 148,
                "diabetes": 1,
                "ejection_fraction": 20,
                "high_blood_pressure": 0,
                "platelets": 162000,
                "serum_creatinine": 1.9,
                "serum_sodium": 136,
                "sex": 1,
                "smoking": 1,
                "time": 30
            }
        ]
        
    },
    "GlobalParameters":{
        "method": "predict"
    }
}

# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())


{'Results': [1]}


TODO: In the cell below, print the logs of the web service and delete the service

In [37]:
from azureml.core.webservice import Webservice
# load existing web service and print logs
service = Webservice(name='best-model-automl-endpoint', workspace=ws)
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)

/bin/bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-02-13T13:38:23,907399500+00:00 - iot-server/run 
2023-02-13T13:38:23,908914100+00:00 - rsyslog/run 
2023-02-13T13:38:23,914673300+00:00 - gunicorn/run 
2023-02-13T13:38:23,921747000+00:00 | gunicorn/run | 
bash: /azureml-envs/azureml_d1809f029ef68d0db142d99e1cd4a92c/lib/libtinfo.so.6: no version information available (required by bash)
2023-02-13T13:38:23,932867000+00:00 | gunicorn/run | ####################################

In [ ]:
#delete webservice
delete_webservice(service)

#delete compute clusters
compute_target.delete()
compute_target.wait_for_completion(show_output=True)
print('compute cluster deleted')

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
